# 马尔可夫决策过程

利用强化学习解决实际问题，第一步要做的事情就是把实际问题抽象为马尔可夫决策过程，也就是明确马尔可夫决策过程的各个组成要素，因此马尔可夫决策过程（Markov decision process，MDP）是强化学习的重要概念。。

## 关键概念与定义

### 马尔可夫性质

当且仅当某时刻的状态只取决于上一时刻的状态时，一个随机过程被称为具有马尔可夫性质（Markov property），用公式表示为。也就是说，当前状态是未来的充分统计量，即下一个状态只取决于当前状态，而不会受到过去状态的影响。需要明确的是，具有马尔可夫性并不代表这个随机过程就和历史完全没有关系。因为虽然时刻的状态只与时刻的状态有关，但是时刻的状态其实包含了时刻的状态的信息，通过这种链式的关系，历史的信息被传递到了现在。马尔可夫性可以大大简化运算，因为只要当前状态可知，所有的历史信息都不再需要了，利用当前状态信息就可以决定未来。

举例子, 我们定义一个状态转移的规律如下: 

|         | Winning | Losing | Drawing |
|---------|---------|--------|---------|
| Winning | 0.6     | 0.1    | 0.3     |
| Losing  | 0.3     | 0.4    | 0.3     |
| Drawing | 0.4     | 0.2    | 0.4     |

> 球队本场state到下场state的概率

如果该球队胜利了，那么两场比赛后的状态如下:
Winning = (0.6 * 0.6) + (0.1 * 0.3) + (0.3 * 0.4) = 0.36 + 0.03 + 0.12 = 0.51
Losing = (0.6 * 0.1) + (0.1 * 0.4) + (0.3 * 0.2) = 0.06 + 0.04 + 0.06 = 0.16
Drawing = (0.6 * 0.3) + (0.1 * 0.3) + (0.3 * 0.4) = 0.18 + 0.03 + 0.12 = 0.33

这样我们就完成了一个符合马尔可夫性质的建模。

### 马尔可夫奖励过程（Markov Reward Process, MRP）

描述了在满足马尔可夫性质的序列中，每个状态附带奖励的数学框架。它是马尔可夫决策过程（MDP）的前置概念，去除了“动作”部分，仅关注状态转移的随机性和奖励的期望。它的建模包含了以下的定义:

| **符号**       | **名称**                | **数学定义**                                                                 |
|----------------|-------------------------|----------------------------------------------------------------------------|
| $\mathcal{S}$  | 状态集合                | 所有可能的状态 $s \in \mathcal{S}$                                        |
| $\mathcal{P}$  | 状态转移概率矩阵        | $\mathcal{P}(s' \| s) = \mathbb{P}(S_{t+1}=s' \| S_t = s)$                |
| $\mathcal{R}$  | 奖励函数                | $\mathcal{R}(s) = \mathbb{E}[R_{t+1} \| S_t = s]$                         |
| $\gamma$       | 折扣因子                | $0 \leq \gamma \leq 1$，控制未来奖励的权重                                |


#### 回报
在符合马尔可夫性质的过程中，所有奖励的衰减之和称为回报，现在为我们的球队的结果设置一个奖励矩阵吧:

|         | Winning | Losing | Drawing |
|---------|---------|--------|---------|
| Winning | +10     | -5     | 0       |
| Losing  | +12      | 0      | +2      |
| Drawing | +5      | -3     | 0       |

如此一来, Winning -> Winning -> Losing 的回报就等于: 10 + （-5） = 5

#### 价值函数

在马尔可夫奖励过程中，一个状态的期望回报（即从这个状态出发的未来累积奖励的期望）被称为这个状态的价值（value）。所有状态的价值就组成了价值函数（value function），价值函数的输入为某个状态，输出为这个状态的价值。我们将价值函数写成: 

$$V(s) = \mathbb{E} \left[ \sum_{k=0}^{\infty} \gamma^k R_{t+k+1} \,\bigg|\, S_t = s \right]$$
- **符号说明**  
  - $\gamma$：折扣因子，$0 \leq \gamma < 1$  
  - $R_{t+k+1}$：时间步 $t+k+1$ 的即时奖励

也可以写作**贝尔曼方程**:

$$V(s) = \underbrace{\sum_{s'} P(s' | s) R(s, s')}_{\text{即时奖励期望-r(s)}} + \gamma \underbrace{\sum_{s'} P(s' | s) V(s')}_{\text{未来价值期望}}$$

- **分解含义**  
  - **第一部分**：从状态 \(s\) 转移到所有可能 \(s'\) 的即时奖励期望。  
  - **第二部分**：转移到后续状态后的未来价值期望，按 $\gamma$ 衰减。






In [15]:
import numpy as np

class MarkovRewardProcess:
    def __init__(self, states, P, R, gamma=0.9):
        """
        初始化马尔可夫奖励过程
        :param states: 状态名称列表，例如 ["Winning", "Losing", "Drawing"]
        :param P: 状态转移概率矩阵，shape=(n_states, n_states)
        :param R: 奖励矩阵，R[s][s']表示从状态s转移到s'的即时奖励
        :param gamma: 折扣因子，默认为0.9
        """
        self.states = states
        self.P = np.array(P)
        self.R = np.array(R)
        self.gamma = gamma
        self.n_states = len(states)
        
        # 验证输入合法性
        assert self.P.shape == (self.n_states, self.n_states), "P矩阵维度不匹配"
        assert self.R.shape == (self.n_states, self.n_states), "R矩阵维度不匹配"
        for row in self.P:
            assert np.isclose(row.sum(), 1.0), "状态转移概率之和必须为1"

    def calculate_value_function(self, max_iter=1000, tol=1e-6):
        """
        通过迭代法求解状态价值函数 V
        :param max_iter: 最大迭代次数
        :param tol: 收敛阈值
        :return: 状态价值向量，V[s]对应状态s的价值
        """
        V = np.zeros(self.n_states)  # 初始化价值函数
        
        for _ in range(max_iter):
            V_new = np.zeros(self.n_states)
            for s in range(self.n_states):
                # 计算每个状态的期望奖励和未来价值
                reward = np.sum(self.P[s] * self.R[s])  # 期望即时奖励 R(s)
                future_value = np.sum(self.P[s] * V)       # 未来价值的期望
                V_new[s] = reward + self.gamma * future_value
                
            if np.max(np.abs(V - V_new)) < tol:
                break
            V = V_new.copy()
        
        return V    
    
    def calculate_value_function_bellman(self):
        """
        通过贝尔曼方程求解状态价值函数 V
        :return: 状态价值向量，V[s]对应状态s的价值
        """
        expected_rewards = np.sum(self.R * self.P, axis=1)
        I = np.eye(self.n_states)
        V_bellman = np.linalg.inv(I - self.gamma * self.P) @ expected_rewards
        return V_bellman

states = ["Winning", "Losing", "Drawing"]

# 状态转移概率矩阵 P
# P[s][s']表示从状态s转移到s'的概率
P = [
    [0.5, 0.2, 0.3],  # Winning -> (Winning, Losing, Drawing)
    [0.4, 0.2, 0.4],  # Losing -> (Winning, Losing, Drawing)
    [0.4, 0.2, 0.4]    # Drawing -> (Winning, Losing, Drawing)
]

# 奖励矩阵 R
# R[s][s']表示从状态s转移到s'的即时奖励
R = [
    [10, -5, 0],   # 从Winning转移到各状态的奖励
    [12, 0, 2],   # 从Losing转移到各状态的奖励
    [5,  -3, 0]    # 从Drawing转移到各状态的奖励
]

# 创建MRP并计算价值函数
mrp = MarkovRewardProcess(states, P, R, gamma=0.9)
V_iterative = mrp.calculate_value_function()
V_bellman = mrp.calculate_value_function_bellman()

# 打印结果
print("状态价值函数 (迭代法):")
for s, name in enumerate(states):
    print(f"  {name}: {V_iterative[s]:.2f}")

print("\n状态价值函数 (贝尔曼方程法):")
for s, name in enumerate(states):
    print(f"  {name}: {V_bellman[s]:.2f}")


状态价值函数 (迭代法):
  Winning: 34.70
  Losing: 36.05
  Drawing: 31.85

状态价值函数 (贝尔曼方程法):
  Winning: 34.70
  Losing: 36.05
  Drawing: 31.85
